In [5]:
import pandas as pd
import numpy as np
import json
import requests
import os

In [43]:
fire_station_forders = os.listdir('../data/fire_station')
fire_station_forders

['전국 119안전센터 현황(2021.12.31)_수정.csv', '전국119안전센터현황(lat_lon).csv']

In [126]:
df_fire = pd.read_csv('../data/fire_station/'+fire_station_forders[0], encoding='CP949',index_col=0).reset_index()
df_fire

,순번,시도본부,소방서,119안전센터명,주소,전화번호
0,1,서울,종로,세종로119안전센터,서울특별시 종로구 율곡로 78(운니동 114-2),02-6981-5740
1,2,서울,종로,신교119안전센터,서울특별시 종로구 자하문로 93 (신교동),02-737-8119
2,3,서울,종로,종로119안전센터,서울특별시 종로구 돈화문로 30 (묘동),02-747-7119
3,4,서울,종로,연건119안전센터,서울특별시 종로구 대학로 91 (연건동),02-747-8119
4,5,서울,종로,신영119안전센터,서울특별시 종로구 진흥로 477 (신영동),02-391-0119
...,...,...,...,...,...,...
1095,1096,창원,마산,양봉119안전센터,경상남도 창원시 마산회원구 자유무역1길 22,055-249-9412
1096,1097,창원,마산,구암119안전센터,경상남도 창원시 마산회원구 팔용로 260,055-249-9420
1097,1098,창원,마산,내서119안전센터,경상남도 창원시 마산회원구 내서읍 중리공단로 104,055-249-9430
1098,1099,창원,마산,호계119안전센터,경상남도 창원시 마산회원구 내서읍 호계본동3길 59-10,055-249-9444


In [127]:
api_key = '79d3d40921cad65ee45ee1059c974972' # daero
# api_key = 'a6bd28c812cc2401914de6d7fcc93171' # jihwan

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['documents'][0]['road_address']
        return float(match_first['x']), float(match_first['y'])
    except:
        return 0, 0

def get_lat_lon(df):
    df.insert(len(df.columns), 'geometry',[np.NaN for k in range(0,len(df))])
    for i in range(0, len(df)):
        try:
            addr = df.iloc[i]['주소']
            x, y = addr_to_lat_lon(addr)
            point = [x, y]
        except:
            point = np.nan

        if x == 0 and y == 0:
            df['geometry'].at[i] = np.nan
        else:
            df['geometry'].at[i] = np.array(point).astype(object)
    

In [128]:
get_lat_lon(df_fire)
df_fire

,순번,시도본부,소방서,119안전센터명,주소,전화번호,geometry
0,1,서울,종로,세종로119안전센터,서울특별시 종로구 율곡로 78(운니동 114-2),02-6981-5740,"[126.987649258831, 37.5768173425423]"
1,2,서울,종로,신교119안전센터,서울특별시 종로구 자하문로 93 (신교동),02-737-8119,"[126.970027499987, 37.5842789170648]"
2,3,서울,종로,종로119안전센터,서울특별시 종로구 돈화문로 30 (묘동),02-747-7119,"[126.992233420661, 37.5712662536965]"
3,4,서울,종로,연건119안전센터,서울특별시 종로구 대학로 91 (연건동),02-747-8119,"[127.001835354449, 37.5794142791836]"
4,5,서울,종로,신영119안전센터,서울특별시 종로구 진흥로 477 (신영동),02-391-0119,"[126.960846444543, 37.6057957808925]"
...,...,...,...,...,...,...,...
1095,1096,창원,마산,양봉119안전센터,경상남도 창원시 마산회원구 자유무역1길 22,055-249-9412,"[128.589000992779, 35.2120223205475]"
1096,1097,창원,마산,구암119안전센터,경상남도 창원시 마산회원구 팔용로 260,055-249-9420,"[128.599298212327, 35.248318329725]"
1097,1098,창원,마산,내서119안전센터,경상남도 창원시 마산회원구 내서읍 중리공단로 104,055-249-9430,"[128.509684452366, 35.2402020292591]"
1098,1099,창원,마산,호계119안전센터,경상남도 창원시 마산회원구 내서읍 호계본동3길 59-10,055-249-9444,"[128.520819830236, 35.2677183649852]"


,순번,시도본부,소방서,119안전센터명,주소,전화번호,geometry
0,1,서울,종로,세종로119안전센터,서울특별시 종로구 율곡로 78(운니동 114-2),02-6981-5740,"[126.987649258831, 37.5768173425423]"
1,2,서울,종로,신교119안전센터,서울특별시 종로구 자하문로 93 (신교동),02-737-8119,"[126.970027499987, 37.5842789170648]"
2,3,서울,종로,종로119안전센터,서울특별시 종로구 돈화문로 30 (묘동),02-747-7119,"[126.992233420661, 37.5712662536965]"
3,4,서울,종로,연건119안전센터,서울특별시 종로구 대학로 91 (연건동),02-747-8119,"[127.001835354449, 37.5794142791836]"
4,5,서울,종로,신영119안전센터,서울특별시 종로구 진흥로 477 (신영동),02-391-0119,"[126.960846444543, 37.6057957808925]"
...,...,...,...,...,...,...,...
1062,1096,창원,마산,양봉119안전센터,경상남도 창원시 마산회원구 자유무역1길 22,055-249-9412,"[128.589000992779, 35.2120223205475]"
1063,1097,창원,마산,구암119안전센터,경상남도 창원시 마산회원구 팔용로 260,055-249-9420,"[128.599298212327, 35.248318329725]"
1064,1098,창원,마산,내서119안전센터,경상남도 창원시 마산회원구 내서읍 중리공단로 104,055-249-9430,"[128.509684452366, 35.2402020292591]"
1065,1099,창원,마산,호계119안전센터,경상남도 창원시 마산회원구 내서읍 호계본동3길 59-10,055-249-9444,"[128.520819830236, 35.2677183649852]"


In [146]:
missing = df_fire[df_fire['geometry'].isna()].index
df_fire.drop(missing, inplace=True)
df_fire.reset_index(inplace=True)
df_fire.drop('index', axis=1, inplace=True)
df_fire

TypeError: reset_index() got an unexpected keyword argument 'dorp'

In [157]:
df_fire.to_csv(path_or_buf='../data/fire_station/전국119안전센터현황(lat_lon).csv', encoding='CP949')